In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
import os

# APIキーをセット (変数名はLangChain側で決められています)
open_api_key = os.environ["openai_api_key"]
serpapi_api_key = os.environ["serpapi_api_key"]

# Agents機能を試す

In [ ]:
# 言語モデルを指定し、ツールをセットアップ
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# エージェントに能力を付与し、Zero-Shot & ReActで機能させることを定義 
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("東京都の人口は、日本の総人口の何パーセントを占めていますか？")

In [ ]:
agent.run("2023年時点の情報で質問に答えて。東京都の人口は、日本の総人口の何パーセントを占めていますか？")

# ユーザ独自のToolsを試す

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

# Serpapiのラッパーをインスタンス化
search = SerpAPIWrapper()

# ツールを定義
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Music Search",
        func=lambda x: "'All I Want For Christmas Is You' by Mariah Carey.", #Mock Function
        description="A Music search engine. Use this more than the normal search if the question is about Music, like 'who is the singer of yesterday?' or 'what is the most popular song in 2022?'",
    )
]

agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("クリスマスに関連する歌で最も有名な曲は？")

# AgentからAPIを叩く

In [ ]:
# Webサーバ (notebook外で実行)
from flask import Flask, request
import hashlib

app = Flask(__name__)

# チャット・メールなどを含む社内情報を収集し、トピックごとに分類されたデータをもっていると仮定。
topics = {
    "プロジェクト管理の効率化" : "チームのプロジェクト管理プロセスを改善し、効率的なタスク管理とスケジュール管理を実現するための施策は、ｘｘｘｘｘｘ",
    "技術トレンドの研究と導入": "最新の技術トレンドを追い、適切な案件においてその導入を検討し、競争力を高める。現状はｙｙｙｙｙｙ",
    "チームメンバーのスキル開発": "チームメンバーのスキル開発を促進し、トレーニングや教育プログラムの導入により技術力と専門知識の向上を図る。現在、ｚｚｚｚｚｚ",
}

@app.route('/api/list_topics', methods=['GET'])
def list_topics():
    return list(topics.keys())

@app.route('/api/get_topic', methods=['POST'])
def get_topic():
    topic_name = request.form.get('topic_name')
    topic = topics[topic_name]
    
    return topic 
     
if __name__ == '__main__':
    app.run()

In [ ]:
!curl localhost:5000/api/list_topics

In [ ]:
!curl -X POST -d "topic_name=プロジェクト管理の効率化" http://localhost:5000/api/get_topic

# API連携するAgent×Tools

In [2]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from typing import Optional, Type, Callable
from pydantic import Field
import requests
import json

# APIキーをセット (変数名はLangChain側で決められています)
from langchain.llms import OpenAI
import os
open_api_key = os.environ["openai_api_key"]

# 言語モデルを指定
llm = OpenAI(temperature=0)

# WebサーバのAPIエンドポイント
url = "http://localhost:5000/api/"


# トピック一覧を取得するツール
class ListTopicTool(BaseTool): # BaseToolクラスのサブクラスとして、クラスを自作
    name = "ListTopic"
    description = """
    Retrieve a list of topics. 
    If a user is requesting a list of topics, Use this functionality.
    For example, the question is about List of Topic, like 'What are the topics for this week?' or 'What kind of topics are available?'"
    """
    
    # エンドポイントにGETリクエストを送信
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        response = requests.get(url + "list_topics")
        topics = response.json()
        return topics
    
    # 非同期実行の関数も必須
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        raise NotImplementedError("ListTopicTool does not support async")

        

def _print_func(text: str) -> None:
    print("\n")
    print(text)

# ユーザにトピック選択を促すツール
class HumanInputTool(BaseTool):
    name = "Human"
    description = """
    If you have obtained topics or if the Human`s desired "topic_name" is ambiguous, 
    you have must ask a Human which topics they are interested in. 
    The input should be a question for the Human. "
    You can also consult a Human when you feel stuck or unsure about what to do next."
    """
    
    # ユーザへの質問を表示する関数
    prompt_func: Callable[[str], None] = Field(default_factory=lambda: _print_func)
    # 入力を受け付ける関数
    input_func: Callable = Field(default_factory=lambda: input)
        
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        # ユーザへの質問を表示する
        self.prompt_func(query)
        # 入力を受け付ける関数を実行
        return self.input_func()

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None,) -> str:
        raise NotImplementedError("Human tool does not support async")

    
# 指定トピックの内容を取得するツール
class GetTopicTool(BaseTool):  # BaseToolクラスのサブクラスとして、クラスを自作
    name = "GetTopic"
    description = """
    Retrieve a specific topic requested by the user.
    This functionality is used when the user is seeking the content of a particular topic, 
    such as when they specify a topic name like "Cloud Expansion Strategy" or when they ask a question like "Tell me about skill development for young employees."
    When executing this functionality, you need to provide input in the form of a dictionary with key-value pairs. 
    The key should be "topic_name" in Japanese and the value should be the specified topic name by the user or the extracted topic name from the user's query.
    """
    
    # エンドポイントにPOSTリクエストを送信
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        query = query.replace("'", '"').encode('utf-8')
        query_dict= json.loads(query)
        response = requests.post(url + "get_topic", query_dict)
        topic_content = response.text
        return topic_content
        
    # 非同期実行の関数も必須
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        raise NotImplementedError("ListTopicTool does not support async")
            
            
# ツールを設定
tools = [
    ListTopicTool(),
    HumanInputTool(),
    GetTopicTool()
]

# エージェントを定義
agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# エージェント起動
agent.run("トピックの一覧を表示し、トピックの内容を把握したいです。")



> Entering new AgentExecutor chain...
 ListTopicを使用してトピックの一覧を取得する必要がある。
Action: ListTopic
Action Input: None
Observation: ['プロジェクト管理の効率化', '技術トレンドの研究と導入', 'チームメンバーのスキル開発']
Thought: ユーザーが求めているトピックを特定する必要がある。
Action: Human
Action Input: ユーザーが求めているトピックを教えてください。

ユーザーが求めているトピックを教えてください。
技術トレンドの研究と導入

Observation: 技術トレンドの研究と導入
Thought: GetTopicを使用して、技術トレンドの研究と導入のトピックを取得する必要がある。
Action: GetTopic
Action Input: {'topic_name': '技術トレンドの研究と導入'}
Observation: 最新の技術トレンドを追い、適切な案件においてその導入を検討し、競争力を高める。現状はｙｙｙｙｙｙ
Thought: 答えを決定する。
Final Answer: 最新の技術トレンドを追い、適切な案件においてその導入を検討し、競争力を高める。現状はｙｙｙｙｙｙ

> Finished chain.


'最新の技術トレンドを追い、適切な案件においてその導入を検討し、競争力を高める。現状はｙｙｙｙｙｙ'

# 外部のサーバに保持されているデータ例

In [ ]:
topics = {
    "プロジェクト管理の効率化" : "チームのプロジェクト管理プロセスを改善し、効率的なタスク管理とスケジュール管理を実現するための施策は、ｘｘｘｘｘｘ",
    "技術トレンドの研究と導入" : "最新の技術トレンドを追い、適切な案件においてその導入を検討し、競争力を高める。現状はｙｙｙｙｙｙ",
    "チームメンバーのスキル開発": "チームメンバーのスキル開発を促進し、トレーニングや教育プログラムの導入により技術力と専門知識の向上を図る。現在、ｚｚｚｚｚｚ",
}

# AgentsがAPI連携してデータを取得する

In [ ]:
agent.run("トピックの一覧を表示して。")

In [ ]:
agent.run("トピックの一覧を表示し、トピックの内容を把握したいです。")

In [ ]:
agent.run("「プロジェクト管理の効率化」について教えて。")

# その他・テスト

In [ ]:
url = "http://localhost:5000/api/"
def get_test():
    response = requests.get(url + "list_topics")
    data = response.json()
    print(data)
    
get_test()

In [ ]:
url = "http://localhost:5000/api/"
def post_test():
    response = requests.post(url + "get_topic", {"topic_name":"プロジェクト管理の効率化"})
    data = response.text
    print(data)
    
post_test()